In [ ]:
from __future__ import print_function
import Baselines_notebook
import cPickle
import numpy as np
import csv
from six.moves import range
import time
from ast import literal_eval
import pdb
import pandas as pd

# read in the data

start = time.time()
with open('/home/alex/Skimlinks/kw_data/one_week_data.txt', 'rt') as f:
    reader = csv.reader(f, delimiter=',')
    data = []
    predictions = []
    count = 1
    for row in reader:
        tmp = []
        # for each session read in with more than one domain id
        this_row = list(literal_eval(','.join(row)))
        ids = [item[1] for item in this_row]
        prev_id = ids[0]
        if len(set(ids))>1:
            data.append(list((count,prev_id)))
            for item in ids[1:]:
                current_id = item
                if current_id != prev_id:
                    data.append(list((count,current_id)))
                    prev_id = current_id
            count += 1
            if count % 100000 == 0:
                print("proccesed {} rows in {} seconds".format(count, time.time() -start))
print(time.time() - start)

In [ ]:
def evaluate_sessions(b, train_data, test_data, predict_last, cut_off = 10, session_key = 'Session_id', domain_key = 'Domain_id'):
    """

    :param b: The baseline to be used
    :param train_data: training sessions
    :param test_data: testing sessions
    :param cut_off: number of domains to view
    :param session_key: session id col name
    :param domain_key: domain id col name
    :return: recall, mrr, accuracy
    """

    test_data.sort_values([session_key],inplace=True)
    unique_domains = train_data[domain_key].unique()
    evaluation_count = 0
    prev_domain_id, prev_sess_id = -1,-1
    recall, mrr, accuracy = 0.0, 0.0, 0.0
    start = time.time()
    for i in range(len(test_data)-1):
        sid = test_data[session_key].values[i]
        nsid = test_data[session_key].values[i+1]
        did = test_data[domain_key].values[i]
        if prev_sess_id != sid:
            prev_sess_id = sid
        else:
            # predicting only the last item reduces the recall by half. 
            if predict_last == True:
                if sid!=nsid:
                    try:
                        predictions = b.predict_domain(sid, prev_domain_id, unique_domains)
                    except:
                        pass
                    predictions[np.isnan(predictions)] = 0
                    predictions += 1e-8 * np.random.rand(len(predictions))
                    
                    try:
                        rank = (predictions > predictions[did]).sum()+1
                    except:
                        rank = cut_off+1
                    assert rank > 0
                    if rank < cut_off:
                        recall +=1
                        mrr += 1.0/rank
                        if rank == 1:
                            accuracy +=1
                    evaluation_count += 1
            else:
                predictions = b.predict_domain(sid, prev_domain_id, unique_domains)
                predictions[np.isnan(predictions)] = 0
                predictions += 1e-8 * np.random.rand(len(predictions))

                try:
                    rank = (predictions > predictions[did]).sum()+1
                except:
                    rank = cut_off+1
                assert rank > 0
                if rank < cut_off:
                    recall +=1
                    mrr += 1.0/rank
                    if rank == 1:
                        accuracy +=1
                evaluation_count += 1                
        prev_domain_id = did
        if i % 1000 == 0:
            print('evaluated', i, "domains with time ", time.time() - start)
    return recall/evaluation_count, mrr/evaluation_count, accuracy/evaluation_count, evaluation_count

tr_data = pd.DataFrame(data[:-350000], columns = ["Session_id", "Domain_id"])
t_data = pd.DataFrame(data[-100000:], columns = ["Session_id", "Domain_id"])
tr_domains = list(tr_data['Domain_id'].unique())
test_domains = list(t_data['Domain_id'].unique())
common_keys = list(set(tr_domains).intersection(test_domains))
t_data = t_data[t_data['Domain_id'].isin(common_keys)]
baselines = [Baselines_notebook.RandomPredections(), Baselines_notebook.Popular(), Baselines_notebook.SessPopular(), Baselines_notebook.domainKNN(), Baselines_notebook.BPR()]

baseline = baselines[2]
baseline.train(tr_data)
Popular_result_2 = evaluate_sessions(baseline, tr_data, t_data, False)

In [ ]:
baseline = baselines[1]
baseline.train(tr_data)
Popular_result_1 = evaluate_sessions(baseline, tr_data, t_data, False)

In [ ]:
baseline = baselines[3]
baseline.train(tr_data)
Popular_result_3 = evaluate_sessions(baseline, tr_data, t_data, False)

In [ ]:
baseline = baselines[4]
baseline.train(tr_data)
Popular_result_4 = evaluate_sessions(baseline, tr_data, t_data, False)